# 📊 Лабораторна робота №2  
## 🧑‍💻 Наука про дані: підготовчий етап  

### 🎯 Мета роботи  
Ознайомлення з основними етапами роботи з даними:  
✅ Від постановки задачі до написання пояснювальної записки.  
✅ Вивчення постановки задачі та природи даних, над якими виконуються аналітичні операції.  

### 📌 Основні поняття  
- **🗃 Сирі дані (raw data)** – початкові, необроблені дані, отримані з різних джерел.  
- **⚙️ Підготовка даних (data preparation)** – процес очищення, перетворення та структурування даних для подальшого аналізу.  

---

👨‍🎓 **ФБ-35**  
✍️ **Яковенко Анастасія**


### Спочатку імпортую необхідні бібліотеки для подальшого використання

In [169]:
from datetime import datetime
import requests
import urllib.request
import os
import pandas as pd
from IPython.display import display
import math

# 📥 Автоматизоване завантаження VHI-індексу  

Для кожної адміністративної одиниці України необхідно завантажити **тестові структуровані файли**, що містять значення **VHI-індексу**.  

✅ **Процедура має бути автоматизована** – параметром передається **індекс (номер) області**.  
✅ **Іменування файлів** – до імені файлу додається **дата та час завантаження**.  
✅ **Обробка повторних запусків** – передбачено:  
   - **Довантаження нових даних** без втрати попередніх.  
   - **Уникнення колізій** при повторному запуску скрипту.  


In [152]:
def download_vhi_data(region_id):
    url = (
        f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/"
        f"get_TS_admin.php?country=UKR&provinceID={region_id}&year1=1981&year2=2020&type=Mean"
    )
    folder = "Files"
    os.makedirs(folder, exist_ok=True)

    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = os.path.join(folder, f"vhi_{region_id}_{timestamp}.csv")

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  
        
        with open(filename, "w", encoding="utf-8") as file:
            file.write(response.text)
        
        print(f" [OK] Дані для регіону {region_id} збережено у файл {filename}")

    except requests.exceptions.RequestException as e:
        print(f" [ERROR] {region_id}: {e}")

if __name__ == "__main__":
    for region_id in range(1, 28):
        download_vhi_data(region_id)


 [OK] Дані для регіону 1 збережено у файл Files\vhi_1_2025-03-10_17-49-43.csv
 [OK] Дані для регіону 2 збережено у файл Files\vhi_2_2025-03-10_17-49-45.csv
 [OK] Дані для регіону 3 збережено у файл Files\vhi_3_2025-03-10_17-49-45.csv
 [OK] Дані для регіону 4 збережено у файл Files\vhi_4_2025-03-10_17-49-46.csv
 [OK] Дані для регіону 5 збережено у файл Files\vhi_5_2025-03-10_17-49-47.csv
 [OK] Дані для регіону 6 збережено у файл Files\vhi_6_2025-03-10_17-49-48.csv
 [OK] Дані для регіону 7 збережено у файл Files\vhi_7_2025-03-10_17-49-49.csv
 [OK] Дані для регіону 8 збережено у файл Files\vhi_8_2025-03-10_17-49-50.csv
 [OK] Дані для регіону 9 збережено у файл Files\vhi_9_2025-03-10_17-49-51.csv
 [OK] Дані для регіону 10 збережено у файл Files\vhi_10_2025-03-10_17-49-51.csv
 [OK] Дані для регіону 11 збережено у файл Files\vhi_11_2025-03-10_17-49-52.csv
 [OK] Дані для регіону 12 збережено у файл Files\vhi_12_2025-03-10_17-49-53.csv
 [OK] Дані для регіону 13 збережено у файл Files\vhi_13_20

# 📥 Завантаження та обробка VHI-даних у DataFrame  

## 🎯 Опис завдання  
Необхідно **зчитати завантажені текстові файли** з індексом **VHI** у **DataFrame** (бібліотека `pandas`).  

## 📌 Вимоги до реалізації  
✔ **Автоматичне зчитування всіх файлів** із вказаної директорії.  
✔ **Коректне форматування даних** при завантаженні у DataFrame.  
✔ **Імена стовпців повинні бути зрозумілими**, без спеціальних символів або пробілів.  
✔ **Окрема процедура**, яка приймає **шлях до директорії** як параметр.   


In [ ]:
import os
import pandas as pd

def reader(directory="Files"):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    all_files = os.listdir(directory)
    dataframe = pd.DataFrame()

    for file in all_files:
        path = os.path.join(directory, file)
        try:
            df = pd.read_csv(path, header=1, names=headers)

            df = df[df["VHI"] != -1]

            df["area"] = file.split("_")[1]

            df["area"] = pd.to_numeric(df["area"].str.strip(), errors="coerce")

            df["Year"] = df["Year"].astype(str).str.replace("<tt><pre>", "", regex=False)
            df = df[~df["Year"].str.contains('</pre></tt>', na=False)]

            df.drop('empty', axis=1, inplace=True, errors="ignore")

            dataframe = pd.concat([dataframe, df], ignore_index=True).drop_duplicates()

        except Exception as e:
            print(f"Error reading file {file}: {e}")

    dataframe["area"] = dataframe["area"].astype(int)
    dataframe = dataframe.sort_values(by=["area", "Year", "Week"]).reset_index(drop=True)

    return dataframe

df = reader("Files")




In [154]:

display(df)

,Year,Week,SMN,SMT,VCI,TCI,VHI,area
0,1982,1.0,0.053,260.31,45.01,39.46,42.23,1
1,1982,2.0,0.054,262.29,46.83,31.75,39.29,1
2,1982,3.0,0.055,263.82,48.13,27.24,37.68,1
3,1982,4.0,0.053,265.33,46.09,23.91,35.00,1
4,1982,5.0,0.050,265.66,41.46,26.65,34.06,1
...,...,...,...,...,...,...,...,...
53401,2020,48.0,0.115,269.40,57.48,31.51,44.49,27
53402,2020,49.0,0.102,267.62,52.70,33.89,43.29,27
53403,2020,50.0,0.095,266.71,50.56,30.20,40.38,27
53404,2020,51.0,0.084,265.81,44.95,26.76,35.86,27


# 🔄 Оновлення індексів областей NOAA на українські  

## 📌 Опис завдання  
Необхідно розробити процедуру, яка замінить індекси областей, використані на порталі **NOAA** (за англійською абеткою), на відповідні українські.  

## 🗂 Відповідність індексів  
| № NOAA | Область            | № Україна | Область         |
|--------|--------------------|-----------|-----------------|
| 1      | Вінницька         | 13        | Миколаївська   |
| 2      | Волинська         | 14        | Одеська        |
| 3      | Дніпропетровська  | 15        | Полтавська     |
| 4      | Донецька          | 16        | Рівненська     |
| 5      | Житомирська       | 17        | Сумська        |
| 6      | Закарпатська      | 18        | Тернопільська  |
| 7      | Запорізька        | 19        | Харківська     |
| 8      | Івано-Франківська | 20        | Херсонська     |
| 9      | Київська          | 21        | Хмельницька    |
| 10     | Кіровоградська    | 22        | Черкаська      |
| 11     | Луганська         | 23        | Чернівецька    |
| 12     | Львівська         | 24        | Чернігівська   |
| 25     | Республіка Крим   | 25        | Республіка Крим |

## 🎯 Вимоги до реалізації  
✔ **Автоматична заміна всіх індексів** у переданому DataFrame.  
✔ **Збереження інших даних без змін**.  
✔ **Перевірка на наявність потрібного стовпця (`Region_ID`)**.  
✔ **Безпечна обробка випадків, коли індексу немає у відповідності**.  

 


In [155]:
area_mapping = {
        1: 'Вінницька облать', 2: 'Волинська область', 3: 'Дніпропетровська область', 4: 'Донецька область',
        5: 'Житомирська область', 6: 'Закарпатська область', 7: 'Запорізька область', 8: 'Івано-Франківська область',
        9: 'Київська область', 10: 'Кіровоградська область', 11: 'Луганська область', 12: 'Львівська область',
        13: 'Миколаївська область', 14: 'Одеська область', 15: 'Полтавська область', 16: 'Рівненська область',
        17: 'Сумська область', 18: 'Тернопільська область', 19: 'Харківська область', 20: 'Херсонська область',
        21: 'Хмельницька область', 22: 'Черкаська область', 23: 'Чернівецька область', 24: 'Чернігівська область',
        25: 'Севастопільська область', 26: 'Київ', 27: 'Крим'
    }
df['area'] = df['area'].replace(area_mapping)

print(df.head())
print(df.tail())

   Year  Week    SMN     SMT    VCI    TCI    VHI              area
0  1982   1.0  0.053  260.31  45.01  39.46  42.23  Вінницька облать
1  1982   2.0  0.054  262.29  46.83  31.75  39.29  Вінницька облать
2  1982   3.0  0.055  263.82  48.13  27.24  37.68  Вінницька облать
3  1982   4.0  0.053  265.33  46.09  23.91  35.00  Вінницька облать
4  1982   5.0  0.050  265.66  41.46  26.65  34.06  Вінницька облать
       Year  Week    SMN     SMT    VCI    TCI    VHI  area
53401  2020  48.0  0.115  269.40  57.48  31.51  44.49  Крим
53402  2020  49.0  0.102  267.62  52.70  33.89  43.29  Крим
53403  2020  50.0  0.095  266.71  50.56  30.20  40.38  Крим
53404  2020  51.0  0.084  265.81  44.95  26.76  35.86  Крим
53405  2020  52.0  0.077  264.70  42.45  27.64  35.05  Крим


In [156]:
display(df)

,Year,Week,SMN,SMT,VCI,TCI,VHI,area
0,1982,1.0,0.053,260.31,45.01,39.46,42.23,Вінницька облать
1,1982,2.0,0.054,262.29,46.83,31.75,39.29,Вінницька облать
2,1982,3.0,0.055,263.82,48.13,27.24,37.68,Вінницька облать
3,1982,4.0,0.053,265.33,46.09,23.91,35.00,Вінницька облать
4,1982,5.0,0.050,265.66,41.46,26.65,34.06,Вінницька облать
...,...,...,...,...,...,...,...,...
53401,2020,48.0,0.115,269.40,57.48,31.51,44.49,Крим
53402,2020,49.0,0.102,267.62,52.70,33.89,43.29,Крим
53403,2020,50.0,0.095,266.71,50.56,30.20,40.38,Крим
53404,2020,51.0,0.084,265.81,44.95,26.76,35.86,Крим


# Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу)

## 1. Ряд VHI для області за вказаний рік
Отримати часовий ряд **VHI** (Value of the Vegetation Health Index) для однієї конкретної області та одного року.  
Наприклад:
- **Вхідні дані**: назва (або індекс) області та рік.
- **Вихід**: набір (DataFrame) значень VHI за кожен тиждень/місяць протягом вказаного року, а також статистичні показники (середнє, максимум, мінімум).

In [157]:
def vhi_series_and_extremes(df, oblast, year):
 
    subset = df[(df["area"] == oblast) & (df["Year"] == year)]
    
    vhi_values = subset["VHI"]
    
    vhi_min = vhi_values.min()
    vhi_max = vhi_values.max()
    
    print(f"Ряд VHI для області {oblast} за {year} рік:")
    print(vhi_values)
    print(f"Мінімальне VHI: {vhi_min}")
    print(f"Максимальне VHI: {vhi_max}")
    
    return subset


vhi_series_and_extremes(df, "Чернігівська область", "2000")


Ряд VHI для області Чернігівська область за 2000 рік:
46400    24.22
46401    27.70
46402    30.68
46403    32.55
46404    34.73
46405    35.08
46406    33.79
46407    34.60
46408    37.70
46409    38.67
46410    38.05
46411    39.32
46412    40.21
46413    41.25
46414    43.66
46415    47.61
46416    52.54
46417    60.14
46418    62.91
46419    63.27
46420    61.76
46421    58.32
46422    56.10
46423    55.32
46424    55.03
46425    54.68
46426    55.27
46427    56.28
46428    57.93
46429    57.94
46430    57.29
46431    56.45
46432    54.95
46433    55.00
46434    55.77
46435    54.21
46436    54.28
46437    52.57
46438    48.01
46439    40.57
46440    33.71
46441    29.41
46442    26.21
46443    18.19
46444    12.26
46445    11.28
46446    11.25
46447    11.38
46448    12.91
46449    14.20
46450    15.07
46451    18.89
Name: VHI, dtype: float64
Мінімальне VHI: 11.25
Максимальне VHI: 63.27


,Year,Week,SMN,SMT,VCI,TCI,VHI,area
46400,2000,1.0,0.023,260.25,8.46,39.97,24.22,Чернігівська область
46401,2000,2.0,0.023,259.38,10.27,45.13,27.70,Чернігівська область
46402,2000,3.0,0.024,259.61,14.76,46.60,30.68,Чернігівська область
46403,2000,4.0,0.027,260.21,19.52,45.57,32.55,Чернігівська область
46404,2000,5.0,0.030,260.59,22.24,47.21,34.73,Чернігівська область
46405,2000,6.0,0.033,262.04,22.48,47.67,35.08,Чернігівська область
46406,2000,7.0,0.036,264.38,22.32,45.26,33.79,Чернігівська область
46407,2000,8.0,0.042,266.69,24.91,44.30,34.60,Чернігівська область
46408,2000,9.0,0.053,268.78,29.61,45.79,37.70,Чернігівська область
46409,2000,10.0,0.065,271.54,31.63,45.70,38.67,Чернігівська область


## 2. Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани
Здійснити пошук:
- **Мінімальних та максимальних значень VHI** у вибраних областях та роках.
- **Середнього** та **медіани** VHI за кожну з обраних областей у вказаних роках.  
Наприклад:
- **Вхідні дані**: список областей, список років.
- **Вихід**: таблиця зі стовпцями \[область, рік, min, max, mean, median\].

In [158]:

def max_vhi(df, area, year):
    
    return df[(df["area"] == area) & (df["Year"] == year)]["VHI"].max()

def min_vhi(df, area, year):
   
    return df[(df["area"] == area) & (df["Year"] == year)]["VHI"].min()

def mean_vhi(df, area, year):
  
    return df[(df["area"] == area) & (df["Year"] == year)]["VHI"].mean()

def median_vhi(df, area, year):
   
    return df[(df["area"] == area) & (df["Year"] == year)]["VHI"].median()



v_min_kyiv_2020 = min_vhi(df, "Київ", "2020")
print("Мінімальний VHI для Києва у 2020 році:", v_min_kyiv_2020)
v_max_kyiv_2020 = max_vhi(df, "Київ", "2020")
print("Максимальний VHI для Києва у 2020 році:", v_max_kyiv_2020)
v_median_kyiv_2020 = median_vhi(df, "Київ", "2020")
print("Медіана VHI для Києва у 2020 році:", v_median_kyiv_2020)
v_mean_kyiv_2020 = mean_vhi(df, "Київ", "2020")
print("Медіана VHI для Києва у 2020 році:", v_mean_kyiv_2020)

Мінімальний VHI для Києва у 2020 році: 18.27
Максимальний VHI для Києва у 2020 році: 66.4
Медіана VHI для Києва у 2020 році: 42.355000000000004
Медіана VHI для Києва у 2020 році: 41.98365384615385


## 3. Ряд VHI за вказаний діапазон років для вказаних областей
Формування об’єднаного ряду VHI для кількох областей у вказаному **діапазоні років** (наприклад, 2000–2010).  
- **Вхідні дані**: список областей, початковий рік, кінцевий рік.
- **Вихід**: зведений DataFrame із усіма VHI-значеннями за обраний період і список статистичних метрик (min, max, mean, median).

In [167]:

def vhi_series_for_regions_year_range(df, regions, start_year, end_year):
   
    
    subset = df[(df["area"].isin(regions)) &
                (df["Year"] >= start_year) &
                (df["Year"] <= end_year)]
    
   
    print(f"Ряд VHI для областей {regions} за період {start_year} - {end_year}:")
    print(subset[["Year", "Week", "area", "VHI"]])
    
    return subset


vhi_series_for_regions_year_range(df, [ "Київська область","Херсонська область"], 2000, 2010)



Ряд VHI для областей ['Київська область', 'Херсонська область'] за період 2000 - 2010:
       Year  Week                area    VHI
16730  2000   1.0    Київська область  34.96
16731  2000   2.0    Київська область  34.07
16732  2000   3.0    Київська область  33.31
16733  2000   4.0    Київська область  32.95
16734  2000   5.0    Київська область  31.87
...     ...   ...                 ...    ...
39035  2010  48.0  Херсонська область  39.27
39036  2010  49.0  Херсонська область  40.88
39037  2010  50.0  Херсонська область  43.83
39038  2010  51.0  Херсонська область  44.61
39039  2010  52.0  Херсонська область  45.57

[1104 rows x 4 columns]


,Year,Week,SMN,SMT,VCI,TCI,VHI,area
16730,2000,1.0,0.042,261.68,16.39,53.54,34.96,Київська область
16731,2000,2.0,0.043,262.64,19.61,48.53,34.07,Київська область
16732,2000,3.0,0.046,263.81,22.66,43.96,33.31,Київська область
16733,2000,4.0,0.048,265.11,24.81,41.09,32.95,Київська область
16734,2000,5.0,0.052,266.89,26.59,37.15,31.87,Київська область
...,...,...,...,...,...,...,...,...
39035,2010,48.0,0.188,283.80,77.36,1.18,39.27,Херсонська область
39036,2010,49.0,0.180,282.29,79.86,1.89,40.88,Херсонська область
39037,2010,50.0,0.176,280.99,85.59,2.08,43.83,Херсонська область
39038,2010,51.0,0.167,279.84,87.38,1.84,44.61,Херсонська область


## 4. Виявлення років з екстремальними посухами для певного відсотка областей
Для **всього набору даних** виявити роки, в яких **екстремальні посухи** (низькі VHI) торкнулися більше вказаного відсотка областей.  
- **Приклад**: якщо 20% областей — це 5 областей із 25 можливих, то потрібно повернути всі роки, коли не менше 5 областей мали екстремально низький VHI.  
- **Вихідні дані**:  
  - **Список років**, у яких виконалася умова.  
  - **Назви областей**, що потрапили під екстремальну посуху.  
  - **Значення VHI** для цих областей.  

In [ ]:

def find_extreme_drought_years(df: pd.DataFrame, vhi_threshold: float, percentage: float) -> pd.DataFrame:
  

    region_year_min = df.groupby(["Year", "area"])["VHI"].min().reset_index()
    region_year_min["extreme"] = region_year_min["VHI"] < vhi_threshold

    total_regions = df["area"].nunique()
    required_count = math.ceil(total_regions * percentage)

    results = []
    for year, group in region_year_min.groupby("Year"):
        extreme_regions = group[group["extreme"]]
        if len(extreme_regions) >= required_count:
            results.append({
                "Year": year,
                "Affected_regions_count": len(extreme_regions),
                "Regions": list(extreme_regions["area"]),
                "VHI_values": list(extreme_regions["VHI"])
            })

    return pd.DataFrame(results)


vhi_threshold = 15         
percentage_threshold = 0.20  
df_result = find_extreme_drought_years(df, vhi_threshold, percentage_threshold)
display(df_result)


,Year,Affected_regions_count,Regions,VHI_values
0,2000,6,"[Івано-Франківська область, Вінницька облать, ...","[9.36, 10.68, 10.6, 6.49, 8.14, 11.25]"
